In [2]:
from __future__ import division
import random
import pprint
import sys
import time
import numpy as np
from optparse import OptionParser
import pickle

from keras import backend as K
from keras.optimizers import Adam, SGD, RMSprop
from keras.layers import Input
from keras.models import Model
from keras_frcnn import config, data_generators
from keras_frcnn import losses as losses
import keras_frcnn.roi_helpers as roi_helpers
from keras.utils import generic_utils
from keras_frcnn.my_simple_parser import get_data
from keras_frcnn import resnet as nn

import os
import tensorflow as tf
os.environ["CUDA_VISIBLE_DEVICES"] = "2"


config1 = tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))
sess = tf.Session(config=config1)



# from keras.backend.tensorflow_backend import set_session
# config = tf.ConfigProto()
# config.gpu_options.per_process_gpu_memory_fraction = 0.3
# set_session(tf.Session(config=config))


sys.setrecursionlimit(40000)
C = config.Config()

C.use_horizontal_flips = False
C.use_vertical_flips = False
C.rot_90 = False
C.model_path = "bishe/model.hdf5"
C.num_rois = 10
C.network = 'resnet50'
C.base_net_weights = "model/resnet50_weights_tf_dim_ordering_tf_kernels.h5"
# else set the path to weights based on backend and model
# C.base_net_weights = nn.get_weight_path()
config_output_filename = "bishe/config.pickle"

epoch_length = 300
num_epochs = 30

all_imgs, classes_count, class_mapping = get_data("/home/ImageProcess/liuqianyu/faster-rcnn-keras-blood/utils/mdb.txt")
with open('/home/ImageProcess/liuqianyu/faster-rcnn-keras-blood/bishe/allimgs.pickle','wb') as f:
    pickle.dump(all_imgs,f)
    print('allimgs.pickle has been written to bishe/allimgs.pickle.')
if 'bg' not in classes_count:
    classes_count['bg'] = 0
    class_mapping['bg'] = len(class_mapping)

C.class_mapping = class_mapping

inv_map = {v: k for k, v in class_mapping.items()}

print('Training images per class:')
pprint.pprint(classes_count)
print('Num classes (including bg) = {}'.format(len(classes_count)))

with open(config_output_filename, 'wb') as config_f:
    pickle.dump(C,config_f)
    print('Config has been written to {}, and can be loaded when testing to ensure correct results'.format(config_output_filename))

random.shuffle(all_imgs)

num_imgs = len(all_imgs)

train_imgs = [s for s in all_imgs if s['imageset'] == 'train']
val_imgs = [s for s in all_imgs if s['imageset'] == 'val']
print('Num train samples {}'.format(len(train_imgs)))
print('Num val samples {}'.format(len(val_imgs)))

data_gen_train = data_generators.get_anchor_gt(train_imgs, classes_count, C, nn.get_img_output_length, K.image_dim_ordering(), mode='train')
data_gen_val = data_generators.get_anchor_gt(val_imgs, classes_count, C, nn.get_img_output_length,K.image_dim_ordering(), mode='val')

if K.image_dim_ordering() == 'th':
    input_shape_img = (3, None, None)
else:
    input_shape_img = (None, None, 3)

img_input = Input(shape=input_shape_img)
roi_input = Input(shape=(None, 4))

# define the base network (resnet here, can be VGG, Inception, etc)
shared_layers = nn.nn_base(img_input, trainable=True)

# define the RPN, built on the base layers
num_anchors = len(C.anchor_box_scales) * len(C.anchor_box_ratios)
rpn = nn.rpn(shared_layers, num_anchors)

classifier = nn.classifier(shared_layers, roi_input, C.num_rois, nb_classes=len(classes_count), trainable=True)

model_rpn = Model(img_input, rpn[:2])
model_classifier = Model([img_input, roi_input], classifier)

# this is a model that holds both the RPN and the classifier, used to load/save weights for the models
model_all = Model([img_input, roi_input], rpn[:2] + classifier)

try:
    print('loading weights from {}'.format(C.base_net_weights))
    model_rpn.load_weights(C.base_net_weights, by_name=True)
    model_classifier.load_weights(C.base_net_weights, by_name=True)
except Exception as e:
    print('Could not load pretrained model weights. Weights can be found in the keras application folder \
        https://github.com/fchollet/keras/tree/master/keras/applications')
    print(e)

optimizer = Adam(lr=1e-5)
optimizer_classifier = Adam(lr=1e-5)
model_rpn.compile(optimizer=optimizer, loss=[losses.rpn_loss_cls(num_anchors), losses.rpn_loss_regr(num_anchors)])
model_classifier.compile(optimizer=optimizer_classifier, loss=[losses.class_loss_cls, losses.class_loss_regr(len(classes_count)-1)], metrics={'dense_class_{}'.format(len(classes_count)): 'accuracy'})
model_all.compile(optimizer='sgd', loss='mae')

iter_num = 0

losses = np.zeros((epoch_length, 5))
rpn_accuracy_rpn_monitor = []
rpn_accuracy_for_epoch = []
start_time = time.time()

best_loss = np.Inf

class_mapping_inv = {v: k for k, v in class_mapping.items()}
print('Starting training')

vis = True

for epoch_num in range(num_epochs):

    progbar = generic_utils.Progbar(epoch_length)
    print('Epoch {}/{}'.format(epoch_num + 1, num_epochs))

    while True:
        try:
            if len(rpn_accuracy_rpn_monitor) == epoch_length and C.verbose:
                mean_overlapping_bboxes = float(sum(rpn_accuracy_rpn_monitor))/len(rpn_accuracy_rpn_monitor)
                rpn_accuracy_rpn_monitor = []
                print('Average number of overlapping bounding boxes from RPN = {} for {} previous iterations'.format(mean_overlapping_bboxes, epoch_length))
                if mean_overlapping_bboxes == 0:
                    print('RPN is not producing bounding boxes that overlap the ground truth boxes. Check RPN settings or keep training.')
            X, Y, img_data = next(data_gen_train)

            loss_rpn = model_rpn.train_on_batch(X, Y)

            P_rpn = model_rpn.predict_on_batch(X)

            R = roi_helpers.rpn_to_roi(P_rpn[0], P_rpn[1], C, K.image_dim_ordering(), use_regr=True, overlap_thresh=0.7, max_boxes=300)
            # note: calc_iou converts from (x1,y1,x2,y2) to (x,y,w,h) format
            X2, Y1, Y2, IouS = roi_helpers.calc_iou(R, img_data, C, class_mapping)

            if X2 is None:
                rpn_accuracy_rpn_monitor.append(0)
                rpn_accuracy_for_epoch.append(0)
                continue

            neg_samples = np.where(Y1[0, :, -1] == 1)
            pos_samples = np.where(Y1[0, :, -1] == 0)

            if len(neg_samples) > 0:
                neg_samples = neg_samples[0]
            else:
                neg_samples = []

            if len(pos_samples) > 0:
                pos_samples = pos_samples[0]
            else:
                pos_samples = []

            rpn_accuracy_rpn_monitor.append(len(pos_samples))
            rpn_accuracy_for_epoch.append((len(pos_samples)))

            if C.num_rois > 1:
                if len(pos_samples) < C.num_rois//2:
                    selected_pos_samples = pos_samples.tolist()
                else:
                    selected_pos_samples = np.random.choice(pos_samples, C.num_rois//2, replace=False).tolist()
                try:
                    selected_neg_samples = np.random.choice(neg_samples, C.num_rois - len(selected_pos_samples), replace=False).tolist()
                except:
                    selected_neg_samples = np.random.choice(neg_samples, C.num_rois - len(selected_pos_samples), replace=True).tolist()

                sel_samples = selected_pos_samples + selected_neg_samples
            else:
            # in the extreme case where num_rois = 1, we pick a random pos or neg sample
                selected_pos_samples = pos_samples.tolist()
                selected_neg_samples = neg_samples.tolist()
                if np.random.randint(0, 2):
                    sel_samples = random.choice(neg_samples)
                else:
                    sel_samples = random.choice(pos_samples)

            loss_class = model_classifier.train_on_batch([X, X2[:, sel_samples, :]], [Y1[:, sel_samples, :], Y2[:, sel_samples, :]])

            losses[iter_num, 0] = loss_rpn[1]
            losses[iter_num, 1] = loss_rpn[2]

            losses[iter_num, 2] = loss_class[1]
            losses[iter_num, 3] = loss_class[2]
            losses[iter_num, 4] = loss_class[3]

            iter_num += 1
            print(iter_num)

            progbar.update(iter_num, [('rpn_cls', np.mean(losses[:iter_num, 0])), ('rpn_regr', np.mean(losses[:iter_num, 1])),
                ('detector_cls', np.mean(losses[:iter_num, 2])), ('detector_regr', np.mean(losses[:iter_num, 3]))])

            if iter_num == epoch_length:
                loss_rpn_cls = np.mean(losses[:, 0])
                loss_rpn_regr = np.mean(losses[:, 1])
                loss_class_cls = np.mean(losses[:, 2])
                loss_class_regr = np.mean(losses[:, 3])
                class_acc = np.mean(losses[:, 4])

                mean_overlapping_bboxes = float(sum(rpn_accuracy_for_epoch)) / len(rpn_accuracy_for_epoch)
                rpn_accuracy_for_epoch = []

                if C.verbose:
                    print('Mean number of bounding boxes from RPN overlapping ground truth boxes: {}'.format(mean_overlapping_bboxes))
                    print('Classifier accuracy for bounding boxes from RPN: {}'.format(class_acc))
                    print('Loss RPN classifier: {}'.format(loss_rpn_cls))
                    print('Loss RPN regression: {}'.format(loss_rpn_regr))
                    print('Loss Detector classifier: {}'.format(loss_class_cls))
                    print('Loss Detector regression: {}'.format(loss_class_regr))
                    print('Elapsed time: {}'.format(time.time() - start_time))

                curr_loss = loss_rpn_cls + loss_rpn_regr + loss_class_cls + loss_class_regr
                iter_num = 0
                start_time = time.time()

                if curr_loss < best_loss:
                    if C.verbose:
                        print('Total loss decreased from {} to {}, saving weights'.format(best_loss,curr_loss))
                    best_loss = curr_loss
                    model_all.save_weights(C.model_path)

                break

        except Exception as e:
            print('Exception: {}'.format(e))
            continue

print('Training complete, exiting.')

Parsing annotation files
119
allimgs.pickle has been written to bishe/allimgs.pickle.
Training images per class:
{'ARCH': 19,
 'ASYM': 15,
 'CALC': 27,
 'CIRC': 24,
 'MISC': 15,
 'SPIC': 19,
 'bg': 0}
Num classes (including bg) = 7
Config has been written to bishe/config.pickle, and can be loaded when testing to ensure correct results
Num train samples 79
Num val samples 16
loading weights from model/resnet50_weights_tf_dim_ordering_tf_kernels.h5
Starting training
Epoch 1/30
1
113/300 [==========>...................] - ETA: 2:36 - rpn_cls: 3.1934 - rpn_regr: 0.5344 - detector_cls: 1.0232 - detector_regr: 0.0825114


224/300 [=====================>........] - ETA: 53s - rpn_cls: 2.6159 - rpn_regr: 0.5209 - detector_cls: 0.8043 - detector_regr: 0.0982Average number of overlapping bounding boxes from RPN = 0.72 for 300 previous iterations
225
228/300 [=====================>........] - ETA: 50s - rpn_cls: 2.6014 - rpn_regr: 0.5201 - detector_cls: 0.8006 - detector_regr: 0.0989229


300/300 [==============================] - 199s 662ms/step - rpn_cls: 2.3896 - rpn_regr: 0.5079 - detector_cls: 0.7426 - detector_regr: 0.1069
Mean number of bounding boxes from RPN overlapping ground truth boxes: 0.7255936675461742
Classifier accuracy for bounding boxes from RPN: 0.9093333341677984
Loss RPN classifier: 1.6888696646073367
Loss RPN regression: 0.45574139966124977
Loss Detector classifier: 0.5405588056376048
Loss Detector regression: 0.13551658595601718
Elapsed time: 198.51578664779663
Total loss decreased from inf to 2.8206864558622082, saving weights
Epoch 2/30
1
 19/300 [>.............................] - ETA: 2:24 - rpn_cls: 1.3518 - rpn_regr: 0.1610 - detector_cls: 0.4776 - detector_regr: 0.1421Exception: a must be non-empty
20
 41/300 [===>..........................] - ETA: 2:14 - rpn_cls: 1.3719 - rpn_regr: 0.2615 - detector_cls: 0.4572 - detector_regr: 0.143342


157/300 [==============>...............] - ETA: 1:15 - rpn_cls: 1.4096 - rpn_regr: 0.3089 - detector_cls: 0.4098 - detector_regr: 0.1504158


207/300 [===================>..........] - ETA: 49s - rpn_cls: 1.3744 - rpn_regr: 0.3141 - detector_cls: 0.3986 - detector_regr: 0.1497Average number of overlapping bounding boxes from RPN = 0.7366666666666667 for 300 previous iterations
208
215/300 [====================>.........] - ETA: 45s - rpn_cls: 1.3674 - rpn_regr: 0.3149 - detector_cls: 0.3968 - detector_regr: 0.1493216


...

133/300 [============>.................] - ETA: 1:10 - rpn_cls: 0.1409 - rpn_regr: 0.0219 - detector_cls: 0.1259 - detector_regr: 0.0514Exception: a must be non-empty
134
151/300 [==============>...............] - ETA: 1:03 - rpn_cls: 0.1444 - rpn_regr: 0.0228 - detector_cls: 0.1233 - detector_regr: 0.0511Exception: a must be non-empty
152
158/300 [==============>...............] - ETA: 1:00 - rpn_cls: 0.1459 - rpn_regr: 0.0230 - detector_cls: 0.1223 - detector_regr: 0.0511Exception: a must be non-empty
159
224/300 [=====================>........] - ETA: 32s - rpn_cls: 0.1537 - rpn_regr: 0.0244 - detector_cls: 0.1167 - detector_regr: 0.0512Exception: a must be non-empty
225
248/300 [=======================>......] - ETA: 22s - rpn_cls: 0.1544 - rpn_regr: 0.0245 - detector_cls: 0.1151 - detector_regr: 0.0513249


252/300 [========================>.....] - ETA: 20s - rpn_cls: 0.1548 - rpn_regr: 0.0245 - detector_cls: 0.1149 - detector_regr: 0.0513Exception: a must be non-empty
Average number of overlapping bounding boxes from RPN = 2.2266666666666666 for 300 previous iterations
253
257/300 [========================>.....] - ETA: 18s - rpn_cls: 0.1553 - rpn_regr: 0.0245 - detector_cls: 0.1146 - detector_regr: 0.0513Exception: a must be non-empty
258
279/300 [==========================>...] - ETA: 8s - rpn_cls: 0.1565 - rpn_regr: 0.0244 - detector_cls: 0.1134 - detector_regr: 0.0512Exception: a must be non-empty
280
300/300 [==============================] - 128s 425ms/step - rpn_cls: 0.1568 - rpn_regr: 0.0242 - detector_cls: 0.1125 - detector_regr: 0.0512
Mean number of bounding boxes from RPN overlapping ground truth boxes: 2.3095975232198143
Classifier accuracy for bounding boxes from RPN: 0.964666662812233
Loss RPN classifier: 0.1659340728115081
Loss RPN regression: 0.02127061996710836
Loss De

101/300 [=========>....................] - ETA: 1:22 - rpn_cls: 0.0144 - rpn_regr: 0.0077 - detector_cls: 0.0677 - detector_regr: 0.0247Exception: a must be non-empty
102
155/300 [==============>...............] - ETA: 1:00 - rpn_cls: 0.0189 - rpn_regr: 0.0070 - detector_cls: 0.0668 - detector_regr: 0.0268Exception: a must be non-empty
156
175/300 [================>.............] - ETA: 52s - rpn_cls: 0.0191 - rpn_regr: 0.0068 - detector_cls: 0.0663 - detector_regr: 0.0272176


236/300 [======================>.......] - ETA: 26s - rpn_cls: 0.0221 - rpn_regr: 0.0063 - detector_cls: 0.0640 - detector_regr: 0.0278Average number of overlapping bounding boxes from RPN = 2.4066666666666667 for 300 previous iterations
237
248/300 [=======================>......] - ETA: 21s - rpn_cls: 0.0226 - rpn_regr: 0.0062 - detector_cls: 0.0635 - detector_regr: 0.0278Exception: a must be non-empty
249
291/300 [============================>.] - ETA: 3s - rpn_cls: 0.0245 - rpn_regr: 0.0059 - detector_cls: 0.0622 - detector_regr: 0.0278292


300/300 [==============================] - 125s 415ms/step - rpn_cls: 0.0247 - rpn_regr: 0.0059 - detector_cls: 0.0620 - detector_regr: 0.0278
Mean number of bounding boxes from RPN overlapping ground truth boxes: 2.39937106918239
Classifier accuracy for bounding boxes from RPN: 0.9823333313067754
Loss RPN classifier: 0.03199617306733245
Loss RPN regression: 0.004269443839219397
Loss Detector classifier: 0.05405600048137908
Loss Detector regression: 0.02709582179644106
Elapsed time: 124.53139925003052
Total loss decreased from 0.13815259135638833 to 0.11741743918437199, saving weights
Training complete, exiting.
